### a.
*Exercise: Build a DNN with 20 hidden layers of 100 neurons each (that's too many, but it's the point of this exercise). Use He initialization and the ELU activation function.*

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import os
import ssl
%load_ext tensorboard

In [2]:
def clear_session():
    keras.backend.clear_session()
    tf.random.set_seed(42)
    np.random.seed(42)

In [3]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense, 
                              kernel_initializer = keras.initializers.he_normal(), 
                              activation = tf.nn.elu)


In [4]:
model = keras.models.Sequential()
model.add(
    keras.layers.Flatten(input_shape = [32, 32, 3]))
for _ in range(20):
    model.add(RegularizedDense(100))



### b.
*Exercise: Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with `keras.datasets.cifar10.load_data()`. The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.*


In [5]:
ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
assert not np.any(np.isnan(X_train_full))

In [8]:
model.add( keras.layers.Dense(10, activation= tf.nn.softmax))

In [9]:
model.compile(optimizer=keras.optimizers.Nadam(lr = 5e-5),
              loss= keras.losses.SparseCategoricalCrossentropy(),
              metrics=["accuracy"],
              )

In [10]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("cifar10_model.h5" ,overwrite=True, save_best_only=True)

run_index = 1
run_logdir = os.path.join(os.curdir, "cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [11]:
%tensorboard --logdir=./cifar10_logs --port=6006


Reusing TensorBoard on port 6006 (pid 8724), started 4 days, 0:22:01 ago. (Use '!kill 8724' to kill it.)

In [12]:
model.fit(X_train, y_train, epochs= 100, verbose = 2, 
          validation_data=(X_valid, y_valid), callbacks = callbacks, workers = 0, batch_size=128)


Epoch 1/100
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
352/352 - 5s - loss: 6.0791 - accuracy: 0.1308 - val_loss: 2.4262 - val_accuracy: 0.1676
Epoch 2/100
352/352 - 5s - loss: 2.2648 - accuracy: 0.1892 - val_loss: 2.1437 - val_accuracy: 0.2178
Epoch 3/100
352/352 - 5s - loss: 2.0947 - accuracy: 0.2323 - val_loss: 2.0509 - val_accuracy: 0.2424
Epoch 4/100
352/352 - 5s - loss: 2.0097 - accuracy: 0.2590 - val_loss: 1.9682 - val_accuracy: 0.2828
Epoch 5/100
352/352 - 4s - loss: 1.9529 - accuracy: 0.2843 - val_loss: 1.9471 - val_accuracy: 0.2826
Epoch 6/100
352/352 - 4s - loss: 1.9094 - accuracy: 0.3030 - val_loss: 1.8773 - val_accuracy: 0.3186
Epoch 7/100
352/352 - 4s - loss: 1.8718 - accuracy: 0.3140 - val_loss: 1.9217 - val_accuracy: 0.2952
Epoch 8/100
352/352 - 4s - loss: 1.8417 - accuracy: 0.3273 - val_loss: 1.9003 - val_accuracy: 0.2996
Epoch 9/100
352/352 - 4s - loss: 1.8109 - accuracy: 0.3397 - val_loss: 1.8204 - val_accuracy: 0.3318
Epoch 10/100
352/35

In [13]:
loaded_model = keras.models.load_model("cifar10_model.h5")
loaded_model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.5425 - accuracy: 0.1108


[1.5424939393997192, 0.11079999804496765]

In [14]:
clear_session()


In [15]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100,kernel_initializer = keras.initializers.he_normal()))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

In [16]:
optimizer = keras.optimizers.Nadam(lr= 5e-4)
model.compile(optimizer = optimizer, loss = keras.losses.SparseCategoricalCrossentropy(), metrics = ["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_stopping_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5",overwrite=True, save_best_only = True)

run_index = 1
run_logdir = os.path.join(os.curdir, "cifar10_log_dir", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
                         
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
          epochs = 100, workers = 0 , 
          callbacks = [early_stopping_cb, model_stopping_cb, tensorboard_cb] ,batch_size=128)

best_model = keras.models.load_model("my_cifar10_bn_model.h5")

best_model.evaluate(X_valid, y_valid)


Epoch 1/100
352/352 [==============================] - 12s 35ms/step - loss: 1.7604 - accuracy: 0.3741 - val_loss: 1.5920 - val_accuracy: 0.4316
Epoch 2/100
352/352 [==============================] - 13s 37ms/step - loss: 1.5374 - accuracy: 0.4551 - val_loss: 1.5532 - val_accuracy: 0.4448
Epoch 3/100
352/352 [==============================] - 13s 36ms/step - loss: 1.4486 - accuracy: 0.4850 - val_loss: 1.4917 - val_accuracy: 0.4694
Epoch 4/100
352/352 [==============================] - 11s 32ms/step - loss: 1.3873 - accuracy: 0.5062 - val_loss: 1.4730 - val_accuracy: 0.4834
Epoch 5/100
352/352 [==============================] - 12s 34ms/step - loss: 1.3348 - accuracy: 0.5264 - val_loss: 1.4279 - val_accuracy: 0.4894
Epoch 6/100
352/352 [==============================] - 12s 33ms/step - loss: 1.2913 - accuracy: 0.5420 - val_loss: 1.4242 - val_accuracy: 0.4936
Epoch 7/100
352/352 [==============================] - 12s 33ms/step - loss: 1.2500 - accuracy: 0.5575 - val_loss: 1.4299 - val_ac

[1.3803691864013672, 0.09120000153779984]

### d.
*Exercise: Try replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).*

In [17]:
clear_session()

In [21]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer = keras.initializers.lecun_normal(), activation=tf.nn.selu))
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))


In [22]:
optimizer = keras.optimizers.Nadam(7e-4)
model.compile(optimizer = optimizer, loss = keras.losses.SparseCategoricalCrossentropy(), metrics = ["accuracy"])

In [23]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5",overwrite=True, save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis = 0)
X_stds = X_train.std(axis = 0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, validation_data=(X_valid_scaled, y_valid),epochs=100, 
          callbacks = callbacks, workers= 0,batch_size=128)

loaded_model = keras.models.load_model("my_cifar10_selu_model.h5")
loaded_model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
352/352 [==============================] - 5s 15ms/step - loss: 1.8765 - accuracy: 0.3294 - val_loss: 1.7422 - val_accuracy: 0.3848
Epoch 2/100
352/352 [==============================] - 5s 14ms/step - loss: 1.6560 - accuracy: 0.4122 - val_loss: 1.6165 - val_accuracy: 0.4300
Epoch 3/100
352/352 [==============================] - 5s 14ms/step - loss: 1.5571 - accuracy: 0.4474 - val_loss: 1.5775 - val_accuracy: 0.4356
Epoch 4/100
352/352 [==============================] - 5s 14ms/step - loss: 1.4794 - accuracy: 0.4770 - val_loss: 1.5460 - val_accuracy: 0.4594
Epoch 5/100
352/352 [==============================] - 5s 13ms/step - loss: 1.4134 - accuracy: 0.5036 - val_loss: 1.5835 - val_accuracy: 0.4486
Epoch 6/100
352/352 [==============================] - 5s 14ms/step - loss: 1.3604 - accuracy: 0.5190 - val_loss: 1.4881 - val_accuracy: 0.4808
Epoch 7/100
352/352 [==============================] - 5s 14ms/step - loss: 1.3081 - accuracy: 0.5389 - val_loss: 1.5410 - val_accuracy:

[1.4637835025787354, 0.10819999873638153]

### e.
*Exercise: Try regularizing the model with alpha dropout. Then, without retraining your model, see if you can achieve better accuracy using MC Dropout.*

In [24]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape = [32, 32, 3]))

for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer = keras.initializers.lecun_normal(), activation=tf.nn.selu))
model.add(keras.layers.AlphaDropout(rate = 0.1))
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

In [25]:

optimizer = keras.optimizers.Nadam(lr=5e-4)
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("cifar10_alpha_dropout_model.h5",overwrite=True, save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "cifar10_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks,batch_size=128)

model = keras.models.load_model("cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
352/352 [==============================] - 5s 15ms/step - loss: 1.9106 - accuracy: 0.3260 - val_loss: 1.7207 - val_accuracy: 0.3972
Epoch 2/100
352/352 [==============================] - 5s 14ms/step - loss: 1.6238 - accuracy: 0.4242 - val_loss: 1.5763 - val_accuracy: 0.4428
Epoch 3/100
352/352 [==============================] - 5s 14ms/step - loss: 1.5157 - accuracy: 0.4643 - val_loss: 1.5571 - val_accuracy: 0.4486
Epoch 4/100
352/352 [==============================] - 4s 13ms/step - loss: 1.4415 - accuracy: 0.4902 - val_loss: 1.5019 - val_accuracy: 0.4838
Epoch 5/100
352/352 [==============================] - 4s 13ms/step - loss: 1.3747 - accuracy: 0.5143 - val_loss: 1.5637 - val_accuracy: 0.4660
Epoch 6/100
352/352 [==============================] - 5s 14ms/step - loss: 1.3181 - accuracy: 0.5341 - val_loss: 1.5202 - val_accuracy: 0.4818
Epoch 7/100
352/352 [==============================] - 5s 14ms/step - loss: 1.2661 - accuracy: 0.5553 - val_loss: 1.5395 - val_accuracy:

[1.4885591268539429, 0.13699999451637268]

In [26]:
class MCAlphaDropout(keras.layers.AlphaDropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [27]:
mc_model = keras.models.Sequential([
    MCAlphaDropout(layer.rate) if isinstance(layer, keras.layers.AlphaDropout) else layer for layer in model.layers
])

In [28]:
def mc_dropout_predict_probas(mc_model, X, n_samples = 10):
    y_probas = [mc_model.predict(X) for _ in range(n_samples)]
    return np.mean(y_probas, axis=0)

def mc_dropout_predict_classes(mc_model, X, n_samples = 10):
    y_probas = mc_dropout_predict_classes(mc_model, X, n_samples)
    return np.argmax(y_probas, axis = 1)

In [29]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

y_pred = mc_dropout_predict_classes(mc_model, X_valid_scaled)
accuracy = np.mean(y_pred == y_valid[:, 0])
accuracy